In [ ]:
import rasterio as rio
import numpy as np
import pandas as pd
import os
import xarray as xr
import rioxarray as rxr
from dask.distributed import Client, LocalCluster
import dask
import hvplot.xarray
from pathlib import Path

In [ ]:
CHUNKS_TLL = {'time': 10, 'lat': 50, 'lon': 50}
CHUNKS_LL = {'lat': 50, 'lon': 50}

In [ ]:
cluster = LocalCluster(n_workers=64, threads_per_worker=1, processes=True, memory_limit='32GB')

client = Client(cluster)
client

In [ ]:
# combine IMERG-Final
from pathlib import Path

imerg_dir = Path(f"../../data-precip-analysis/imerg_final/ss_method_nc")
imerg_fns = imerg_dir.glob("*.nc")

imerg_ds = xr.open_mfdataset(imerg_fns, chunks=CHUNKS_TLL, combine='by_coords')
imerg_ds

In [ ]:
imerg_ds['precip'].sel(time=slice('2017-01-01', '2017-12-31')).hvplot(groupby='time')

In [ ]:
precip = imerg_ds['precip'].transpose('time', 'lat', 'lon').chunk(CHUNKS_TLL)
if 'spatial_ref' in precip.coords:
    precip = precip.drop('spatial_ref')
precip

In [ ]:
existing_ds_fn = Path("../../data-cumberland/tennessee/basins/cumberland/pre_processing/nc/combined_data.nc")
existing_ds = xr.open_dataset(existing_ds_fn)
existing_ds

In [ ]:
new_ds = existing_ds.copy(deep=True)

new_ds['precip'] = precip
new_ds

In [ ]:
precip.sel(time='2017-01-01').hvplot()

In [ ]:
new_ds['precip'].sel(time='2017-01-01').hvplot()

In [ ]:
dst_fp = Path('../../data-precip-analysis/tennessee/basins/cumberland/pre_processing/nc/combined_data.nc')
new_ds.to_netcdf(dst_fp)

# Read combined data

In [ ]:
res_ds = xr.open_dataset("/water2/pdas47/2023_01_24-river-regulation/data-precip-analysis/tennessee/basins/cumberland/pre_processing/nc/combined_data.nc")
res_ds

In [ ]:
from matplotlib import pyplot as plt

f, ax = plt.subplots(figsize=(10, 4))

res_ds['precip'].sel(time='2017-01-02', method='nearest').plot(x='lon', y='lat', ax=ax)
ax.set_aspect('equal')

In [ ]:
from rat.run_rat import run_rat
from ruamel.yaml import YAML

config_fn = Path("../../params/rat-precip-analysis.yaml")

config_yaml = YAML()
config_yaml = config_yaml.load(config_fn.read_text())

start_date = config_yaml['BASIN']['start']
end_date = config_yaml['BASIN']['end']
# run_rat(config_fn)

In [ ]:
import datetime

def generate_state_and_inputs(forcings_startdate, forcings_enddate, combined_datapath, out_dir):
    combined_datapath = Path(combined_datapath)
    assert combined_datapath.exists()
    out_dir = Path(out_dir)
    out_dir.mkdir(exist_ok=True, parents=True)

    # Generate state. Assuming `nc_fmt_data` contains all the data, presumably containing enough data
    # to create state file (upto 90 days prior data from forcings_startdate)
    combined_data = xr.open_dataset(combined_datapath)

    state_startdate = forcings_startdate - datetime.timedelta(days=90)
    state_enddate = forcings_startdate - datetime.timedelta(days=1)

    state_ds = combined_data.sel(time=slice(state_startdate, state_enddate))
    state_outpath = out_dir / "state.nc"
    print(f"Saving state at: {state_outpath}")
    state_ds.drop('spatial_ref').to_netcdf(state_outpath)

    # Generate the metsim input
    forcings_ds = combined_data.sel(time=slice(forcings_startdate, forcings_enddate))
    forcings_outpath = out_dir / "metsim_input.nc"
    print(f"Saving forcings: {forcings_outpath}")
    forcings_ds.drop('spatial_ref').to_netcdf(forcings_outpath)

    return state_outpath, forcings_outpath

In [ ]:
generate_state_and_inputs(
    start_date, 
    end_date, 
    "/water2/pdas47/2023_01_24-river-regulation/data-precip-analysis/tennessee/basins/cumberland/pre_processing/nc/combined_data.nc", 
    "/water2/pdas47/2023_01_24-river-regulation/data-precip-analysis/tennessee/basins/cumberland/metsim/metsim_inputs"
)

Run RAT from step 5 to generate inflow.